# Using Hugging Face With Skills

In this notebook, we demonstrate using Hugging Face models for Skills using both SemanticMemory and text completions. 

SK supports downloading models from the Hugging Face that can perform the following tasks: text-generation, text2text-generation, summarization, and sentence-similarity. You can search for models by task at https://huggingface.co/models.

In [1]:
!python -m pip install semantic-kernel==0.5.1.dev0
!python -m pip install sentence-transformers==2.3.1

In [1]:
import semantic_kernel as sk
from ov_pipeline.ov_text_completion import OpenVINOTextCompletion
from ov_pipeline.ov_text_embedding import OpenVINOTextEmbedding

First, we will create a kernel and add both text completion and embedding services. 

For text completion, we are choosing GPT2. This is a text-generation model. (Note: text-generation will repeat the input in the output, text2text-generation will not.)
For embeddings, we are using sentence-transformers/all-MiniLM-L6-v2. Vectors generated for this model are of length 384 (compared to a length of 1536 from OpenAI ADA).

The following step may take a few minutes when run for the first time as the models will be downloaded to your local machine.

In [15]:
kernel = sk.Kernel()
ov_config = {"PERFORMANCE_HINT": "LATENCY",
             "NUM_STREAMS": "1", "CACHE_DIR": ""}
# Configure LLM service
kernel.add_text_completion_service(
    service_id="gpt2",
    service=OpenVINOTextCompletion(ai_model_id="./llm_model", task="text-generation", model_kwargs={
                                   "device": "CPU", "ov_config": ov_config}, pipeline_kwargs={"max_new_tokens": 64})
)
kernel.add_text_embedding_generation_service(
    service_id="sentence-transformers/all-MiniLM-L6-v2",
    service=OpenVINOTextEmbedding(ai_model_id="./embedding_model"),
)
kernel.register_memory_store(memory_store=sk.memory.VolatileMemoryStore())
kernel.import_plugin(sk.core_plugins.TextMemoryPlugin(), "openvino")

Compiling the model to CPU ...
device must be of type <class 'str'> but got <class 'torch.device'> instead


KernelPlugin(name='openvino', description=None, functions={'recall': KernelFunction(plugin_name='openvino', description='Recall a fact from the long term memory', name='recall', is_semantic=False, stream_function=<bound method TextMemoryPlugin.recall of TextMemoryPlugin()>, parameters=[ParameterView(name='input', description='The information to retrieve', default_value='', type_='string', required=False), ParameterView(name='limit', description='The maximum number of relevant memories to recall.', default_value='1', type_='string', required=False), ParameterView(name='relevance', description='The relevance score, from 0.0 to 1.0; 1.0 means perfect match', default_value='0.75', type_='string', required=False), ParameterView(name='collection', description='The collection to search for information', default_value='generic', type_='string', required=False)], delegate_type=<DelegateTypes.InStringAndContextOutTaskString: 13>, function=<bound method TextMemoryPlugin.recall of TextMemoryPlugin

### Add Memories and Define a skill to use them

Most models available on huggingface.co are not as powerful as OpenAI GPT-3+. Your skills will likely need to be simpler to accommodate this.

In [16]:
await kernel.memory.save_information(collection="animal-facts", id="info1", text="Sharks are fish.")
await kernel.memory.save_information(collection="animal-facts", id="info2", text="Whales are mammals.")
await kernel.memory.save_information(collection="animal-facts", id="info3", text="Penguins are birds.")
await kernel.memory.save_information(collection="animal-facts", id="info4", text="Dolphins are mammals.")
await kernel.memory.save_information(collection="animal-facts", id="info5", text="Flies are insects.")

# Define semantic function using SK prompt template language
my_prompt = """I know these animal facts: {{recall $query1}} {{recall $query2}} {{recall $query3}} and """

# Create the semantic function
my_function = kernel.create_semantic_function(prompt_template=my_prompt, max_tokens=45, temperature=0.5, top_p=0.5)

Let's now see what the completion looks like! Remember, "gpt2" is nowhere near as large as ChatGPT, so expect a much simpler answer.

In [18]:
context = kernel.create_new_context()
context[sk.core_plugins.TextMemoryPlugin.COLLECTION_PARAM] = "animal-facts"
context[sk.core_plugins.TextMemoryPlugin.RELEVANCE_PARAM] = "0.3"

context["query1"] = "animal that swims"
context["query2"] = "animal that flies"
context["query3"] = "penguins are?"
output = await kernel.run(my_function, input_vars=context.variables)

output = str(output).strip()

query_result1 = await kernel.memory.search(
    collection="animal-facts", query=context["query1"], limit=1, min_relevance_score=0.3
)
query_result2 = await kernel.memory.search(
    collection="animal-facts", query=context["query2"], limit=1, min_relevance_score=0.3
)
query_result3 = await kernel.memory.search(
    collection="animal-facts", query=context["query3"], limit=1, min_relevance_score=0.3
)

print(f"gpt2 completed prompt with: '{output}'")

/home/ethan/intel/semantic-kernel.openvino/openvino_env/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:407: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/ethan/intel/semantic-kernel.openvino/openvino_env/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:412: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  w

gpt2 completed prompt with: 'I know these animal facts: ["Dolphins are mammals."] ["Flies are insects."] ["Penguins are birds."] and  "Horses are mammals."
The following is a list of the most common and most common species of animals in the world.
1. The "Dolphin"
The dolphin is a large, round, black, dolphin-like creature. It is a large, round, black, dolphin-like creature'
